In [1]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))
    
from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data, rel_disp, third_moment
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.show_plot import show_plot
import numpy as np
import matplotlib

In [3]:
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, opt_set, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=opt_set
    )
    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 16})
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()):
            plot_data = results[coord]['numerical'][opts]
            m_diff, n_diff = [], []
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                    
                    
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'",""],
                            [": True",""],
                            ["_", " "],
                            ["{",""],["}",""],[","," "]]                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    linewidth = linewidth,
                    mnorm=mnorm, color = colors[opt_i]
                )
                
                dp_dr = psi_coord.dx_dr(results[coord]['grid']['r'])
                numeric_rel_d = rel_disp(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord)
                analytic_rel_d = rel_disp(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord)
#                 print("analytic rd", analytic_rel_d)
#                 print("numeric rd", numeric_rel_d)
                dispersion_ratio = (numeric_rel_d / analytic_rel_d - 1)*100
                print(f"dispersion ratio excess {opts}: {dispersion_ratio.to(setup.si.dimensionless).magnitude:.2g} %")
                print("\n")
                                
                numeric_mass = third_moment(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord, normalised=False)
                analytic_mass = third_moment(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord, normalised=False)
#                 print("analytic mass", analytic_mass * 4/3 * np.pi * setup.rho_w/setup.rho_a)
#                 print("numeric mass", numeric_mass * 4/3 * np.pi * setup.rho_w/setup.rho_a)
                mass_ratio = (numeric_mass / analytic_mass - 1) * 100
                print(f"mass ratio excess: {mass_ratio.to(setup.si.dimensionless).magnitude:.2g} %")
                print("\n")
        print(f"grid_layout: {coord}")
        show_plot(filename)

In [4]:
plot_setup = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(r0=1, base=2), "GC_max":.26,"n_bins":75}
plot_setup2 = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07,"n_bins":75}

In [5]:
compute_and_plot(**plot_setup, opt_set=({"n_iters":1},), plots=['n','m'], filename = 'fig_upwinda.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    3.8s finished


dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


dispersion ratio excess upwind: 24 %


mass ratio excess: 5.5 %


dispersion ratio excess upwind: 71 %


mass ratio excess: 4.4 %


grid_layout: x_log_of_pn


In [6]:
compute_and_plot(**plot_setup2, opt_set=({"n_iters":1},), plots=['n','m'],  filename = 'fig_upwindb.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    4.2s finished


dispersion ratio excess upwind: 0 %


mass ratio excess: 0 %


dispersion ratio excess upwind: 19 %


mass ratio excess: 0.89 %


dispersion ratio excess upwind: 48 %


mass ratio excess: 0.41 %


grid_layout: x_id


In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters':2, 'infinite_gauge':True, 'flux_corrected_transport':True, 'DPDC':True}, {'n_iters':2, 'infinite_gauge':True, 'flux_corrected_transport':True, 'DPDC':True, 'third_order_terms': True}), plots=['n'], filename = 'fig_DPDC.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters':3, 'infinite_gauge':True, 'flux_corrected_transport':True, 'third_order_terms':True}, {'n_iters':2, 'infinite_gauge':True, 'flux_corrected_transport':True, 'DPDC':True}), plots=['n'], filename = 'fig_DPDC.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters':3, 'infinite_gauge':True, 'flux_corrected_transport':True, 'third_order_terms':True}, {'n_iters':3, 'infinite_gauge':True, 'flux_corrected_transport':True, 'DPDC':True}), plots=['n'], filename = 'fig_DPDC.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters':3, 'infinite_gauge':True, 'flux_corrected_transport':True, 'third_order_terms':True}, {'n_iters':3, 'infinite_gauge':True, 'flux_corrected_transport':True, 'third_order_terms':True,'DPDC':True}), plots=['n'], filename = 'fig_DPDC.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':3}), plots=['n'], filename = 'fig_mpdatas.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True}), plots=['n'], filename = 'fig_iga.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':3},{'n_iters':3,'third_order_terms':True}), plots=['n'], filename = 'fig_tot.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
), plots=['n'], filename='fig_multiopt.pdf')